In [1]:
import pickle, random, copy, platform, os, time
import numpy as np
import tensorflow as tf
import email_sending
import tflearn
import tflearn.activations as activations
from PIL import Image, ImageDraw
from __future__ import division, print_function, absolute_import
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.activations import relu
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.conv import avg_pool_2d, conv_2d, max_pool_2d
from tflearn.layers.core import dropout, flatten, fully_connected, input_data
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import batch_normalization
from tflearn.utils import repeat

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
def read_data():
    with open('resnet_data', 'rb') as f:
        data = pickle.load(f)
        Xtr = data[0]
        Xtr = np.reshape(Xtr, (-1,75,75,2))
        Ytr = data[1]
        Xte =data[2]
        Xte = np.reshape(Xte, (-1,75,75,2))
        rf =data[3]
    return Xtr, Ytr, Xte, rf

# expresses the label data in one-hot encoding.
def onehot_encoding (Y, y_class):
    Y_onehot = np.zeros((Y.size, int(y_class)))
    for i in range(Y.size):
        Y_onehot[i][Y[i]] = 1
    return Y_onehot

In [3]:
def resnet32(x, classes, n = 5): #classes =2
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, n, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, n - 1, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, n - 1, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')

    return net

In [4]:
def resnext(x, classes, n =5):
    net = tflearn.conv_2d(x, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.resnext_block(net, n, 16, 32)
    net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
    net = tflearn.resnext_block(net, n-1, 32, 32)
    net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
    net = tflearn.resnext_block(net, n-1, 64, 32)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    net = tflearn.fully_connected(net, classes, activation='softmax')
    
    return net

In [5]:
def vgg(x, classes):
    
    network = conv_2d(x, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 128, 3, activation='relu')
    network = conv_2d(network, 128, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, classes, activation='softmax')
    
    return network

In [6]:
def block35(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None,name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 32, 3, bias=False, activation=None,name='Conv2d_0b_3x3')))
    tower_conv2_0 = relu(batch_normalization(conv_2d(net, 32, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2_0, 48,3, bias=False, activation=None, name='Conv2d_0b_3x3')))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 64,3, bias=False, activation=None, name='Conv2d_0c_3x3')))
    tower_mixed = merge([tower_conv, tower_conv1_1, tower_conv2_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net

def block17(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv_1_0 = relu(batch_normalization(conv_2d(net, 128, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv_1_1 = relu(batch_normalization(conv_2d(tower_conv_1_0, 160,[1,7], bias=False, activation=None,name='Conv2d_0b_1x7')))
    tower_conv_1_2 = relu(batch_normalization(conv_2d(tower_conv_1_1, 192, [7,1], bias=False, activation=None,name='Conv2d_0c_7x1')))
    tower_mixed = merge([tower_conv,tower_conv_1_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net


def block8(net, scale=1.0, activation="relu"):
    tower_conv = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_1x1')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 192, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 224, [1,3], bias=False, activation=None, name='Conv2d_0b_1x3')))
    tower_conv1_2 = relu(batch_normalization(conv_2d(tower_conv1_1, 256, [3,1], bias=False, name='Conv2d_0c_3x1')))
    tower_mixed = merge([tower_conv,tower_conv1_2], mode='concat', axis=3)
    tower_out = relu(batch_normalization(conv_2d(tower_mixed, net.get_shape()[3], 1, bias=False, activation=None, name='Conv2d_1x1')))
    net += scale * tower_out
    if activation:
        if isinstance(activation, str):
            net = activations.get(activation)(net)
        elif hasattr(activation, '__call__'):
            net = activation(net)
        else:
            raise ValueError("Invalid Activation.")
    return net

In [7]:
def inception_resnet_v2(x, classes):

    dropout_keep_prob = 0.8

    conv1a_3_3 = relu(batch_normalization(conv_2d(x, 32, 3, strides=2, bias=False, padding='VALID',activation=None,name='Conv2d_1a_3x3')))
    conv2a_3_3 = relu(batch_normalization(conv_2d(conv1a_3_3, 32, 3, bias=False, padding='VALID',activation=None, name='Conv2d_2a_3x3')))
    conv2b_3_3 = relu(batch_normalization(conv_2d(conv2a_3_3, 64, 3, bias=False, activation=None, name='Conv2d_2b_3x3')))
    maxpool3a_3_3 = max_pool_2d(conv2b_3_3, 3, strides=2, padding='VALID', name='MaxPool_3a_3x3')
    conv3b_1_1 = relu(batch_normalization(conv_2d(maxpool3a_3_3, 80, 1, bias=False, padding='VALID',activation=None, name='Conv2d_3b_1x1')))
    conv4a_3_3 = relu(batch_normalization(conv_2d(conv3b_1_1, 192, 3, bias=False, padding='VALID',activation=None, name='Conv2d_4a_3x3')))
    maxpool5a_3_3 = max_pool_2d(conv4a_3_3, 3, strides=2, padding='VALID', name='MaxPool_5a_3x3')

    tower_conv = relu(batch_normalization(conv_2d(maxpool5a_3_3, 96, 1, bias=False, activation=None, name='Conv2d_5b_b0_1x1')))

    tower_conv1_0 = relu(batch_normalization(conv_2d(maxpool5a_3_3, 48, 1, bias=False, activation=None, name='Conv2d_5b_b1_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 64, 5, bias=False, activation=None, name='Conv2d_5b_b1_0b_5x5')))

    tower_conv2_0 = relu(batch_normalization(conv_2d(maxpool5a_3_3, 64, 1, bias=False, activation=None, name='Conv2d_5b_b2_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2_0, 96, 3, bias=False, activation=None, name='Conv2d_5b_b2_0b_3x3')))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 96, 3, bias=False, activation=None,name='Conv2d_5b_b2_0c_3x3')))

    tower_pool3_0 = avg_pool_2d(maxpool5a_3_3, 3, strides=1, padding='same', name='AvgPool_5b_b3_0a_3x3')
    tower_conv3_1 = relu(batch_normalization(conv_2d(tower_pool3_0, 64, 1, bias=False, activation=None,name='Conv2d_5b_b3_0b_1x1')))

    tower_5b_out = merge([tower_conv, tower_conv1_1, tower_conv2_2, tower_conv3_1], mode='concat', axis=3)

    net = repeat(tower_5b_out, 10, block35, scale=0.17)

    tower_conv = relu(batch_normalization(conv_2d(net, 384, 3, bias=False, strides=2,activation=None, padding='VALID', name='Conv2d_6a_b0_0a_3x3')))
    tower_conv1_0 = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, activation=None, name='Conv2d_6a_b1_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1_0, 256, 3, bias=False, activation=None, name='Conv2d_6a_b1_0b_3x3')))
    tower_conv1_2 = relu(batch_normalization(conv_2d(tower_conv1_1, 384, 3, bias=False, strides=2, padding='VALID', activation=None,name='Conv2d_6a_b1_0c_3x3')))
    tower_pool = max_pool_2d(net, 3, strides=2, padding='VALID',name='MaxPool_1a_3x3')
    net = merge([tower_conv, tower_conv1_2, tower_pool], mode='concat', axis=3)
    net = repeat(net, 20, block17, scale=0.1)

    tower_conv = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, activation=None, name='Conv2d_0a_1x1')))
    tower_conv0_1 = relu(batch_normalization(conv_2d(tower_conv, 384, 3, bias=False, strides=2, padding='VALID', activation=None,name='Conv2d_0a_1x1')))

    tower_conv1 = relu(batch_normalization(conv_2d(net, 256, 1, bias=False, padding='VALID', activation=None,name='Conv2d_0a_1x1')))
    tower_conv1_1 = relu(batch_normalization(conv_2d(tower_conv1,288,3, bias=False, strides=2, padding='VALID',activation=None, name='COnv2d_1a_3x3')))

    tower_conv2 = relu(batch_normalization(conv_2d(net, 256,1, bias=False, activation=None,name='Conv2d_0a_1x1')))
    tower_conv2_1 = relu(batch_normalization(conv_2d(tower_conv2, 288,3, bias=False, name='Conv2d_0b_3x3',activation=None)))
    tower_conv2_2 = relu(batch_normalization(conv_2d(tower_conv2_1, 320, 3, bias=False, strides=2, padding='VALID',activation=None, name='Conv2d_1a_3x3')))

    tower_pool = max_pool_2d(net, 3, strides=2, padding='VALID', name='MaxPool_1a_3x3')
    net = merge([tower_conv0_1, tower_conv1_1,tower_conv2_2, tower_pool], mode='concat', axis=3)

    net = repeat(net, 9, block8, scale=0.2)
    net = block8(net, activation=None)

    net = relu(batch_normalization(conv_2d(net, 1536, 1, bias=False, activation=None, name='Conv2d_7b_1x1')))
    net = avg_pool_2d(net, net.get_shape().as_list()[1:3],strides=2, padding='VALID', name='AvgPool_1a_8x8')
    net = flatten(net)
    net = dropout(net, dropout_keep_prob)
    loss = fully_connected(net, classes,activation='softmax')
    
    return loss

In [8]:
def googlenet(x, classes):
    conv1_7_7 = conv_2d(x, 64, 7, strides=2, activation='relu', name='conv1_7_7_s2')
    pool1_3_3 = max_pool_2d(conv1_7_7, 3, strides=2)
    pool1_3_3 = local_response_normalization(pool1_3_3)
    conv2_3_3_reduce = conv_2d(pool1_3_3, 64, 1, activation='relu', name='conv2_3_3_reduce')
    conv2_3_3 = conv_2d(conv2_3_3_reduce, 192, 3, activation='relu', name='conv2_3_3')
    conv2_3_3 = local_response_normalization(conv2_3_3)
    pool2_3_3 = max_pool_2d(conv2_3_3, kernel_size=3, strides=2, name='pool2_3_3_s2')

    # 3a
    inception_3a_1_1 = conv_2d(pool2_3_3, 64, 1, activation='relu', name='inception_3a_1_1')
    inception_3a_3_3_reduce = conv_2d(pool2_3_3, 96, 1, activation='relu', name='inception_3a_3_3_reduce')
    inception_3a_3_3 = conv_2d(inception_3a_3_3_reduce, 128, filter_size=3,  activation='relu', name='inception_3a_3_3')
    inception_3a_5_5_reduce = conv_2d(pool2_3_3, 16, filter_size=1, activation='relu', name='inception_3a_5_5_reduce')
    inception_3a_5_5 = conv_2d(inception_3a_5_5_reduce, 32, filter_size=5, activation='relu', name='inception_3a_5_5')
    inception_3a_pool = max_pool_2d(pool2_3_3, kernel_size=3, strides=1, name='inception_3a_pool')
    inception_3a_pool_1_1 = conv_2d(inception_3a_pool, 32, filter_size=1, activation='relu', name='inception_3a_pool_1_1')
    inception_3a_output = merge([inception_3a_1_1, inception_3a_3_3, inception_3a_5_5, inception_3a_pool_1_1], mode='concat', axis=3)

    # 3b
    inception_3b_1_1 = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_1_1')
    inception_3b_3_3_reduce = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_3_3_reduce')
    inception_3b_3_3 = conv_2d(inception_3b_3_3_reduce, 192, filter_size=3, activation='relu', name='inception_3b_3_3')
    inception_3b_5_5_reduce = conv_2d(inception_3a_output, 32, filter_size=1, activation='relu', name='inception_3b_5_5_reduce')
    inception_3b_5_5 = conv_2d(inception_3b_5_5_reduce, 96, filter_size=5,  name='inception_3b_5_5')
    inception_3b_pool = max_pool_2d(inception_3a_output, kernel_size=3, strides=1,  name='inception_3b_pool')
    inception_3b_pool_1_1 = conv_2d(inception_3b_pool, 64, filter_size=1, activation='relu', name='inception_3b_pool_1_1')
    inception_3b_output = merge([inception_3b_1_1, inception_3b_3_3, inception_3b_5_5, inception_3b_pool_1_1], mode='concat', axis=3, name='inception_3b_output')
    pool3_3_3 = max_pool_2d(inception_3b_output, kernel_size=3, strides=2, name='pool3_3_3')

    # 4a
    inception_4a_1_1 = conv_2d(pool3_3_3, 192, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4a_3_3_reduce = conv_2d(pool3_3_3, 96, filter_size=1, activation='relu', name='inception_4a_3_3_reduce')
    inception_4a_3_3 = conv_2d(inception_4a_3_3_reduce, 208, filter_size=3,  activation='relu', name='inception_4a_3_3')
    inception_4a_5_5_reduce = conv_2d(pool3_3_3, 16, filter_size=1, activation='relu', name='inception_4a_5_5_reduce')
    inception_4a_5_5 = conv_2d(inception_4a_5_5_reduce, 48, filter_size=5,  activation='relu', name='inception_4a_5_5')
    inception_4a_pool = max_pool_2d(pool3_3_3, kernel_size=3, strides=1,  name='inception_4a_pool')
    inception_4a_pool_1_1 = conv_2d(inception_4a_pool, 64, filter_size=1, activation='relu', name='inception_4a_pool_1_1')
    inception_4a_output = merge([inception_4a_1_1, inception_4a_3_3, inception_4a_5_5, inception_4a_pool_1_1], mode='concat', axis=3, name='inception_4a_output')

    # 4b
    inception_4b_1_1 = conv_2d(inception_4a_output, 160, filter_size=1, activation='relu', name='inception_4a_1_1')
    inception_4b_3_3_reduce = conv_2d(inception_4a_output, 112, filter_size=1, activation='relu', name='inception_4b_3_3_reduce')
    inception_4b_3_3 = conv_2d(inception_4b_3_3_reduce, 224, filter_size=3, activation='relu', name='inception_4b_3_3')
    inception_4b_5_5_reduce = conv_2d(inception_4a_output, 24, filter_size=1, activation='relu', name='inception_4b_5_5_reduce')
    inception_4b_5_5 = conv_2d(inception_4b_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4b_5_5')
    inception_4b_pool = max_pool_2d(inception_4a_output, kernel_size=3, strides=1,  name='inception_4b_pool')
    inception_4b_pool_1_1 = conv_2d(inception_4b_pool, 64, filter_size=1, activation='relu', name='inception_4b_pool_1_1')
    inception_4b_output = merge([inception_4b_1_1, inception_4b_3_3, inception_4b_5_5, inception_4b_pool_1_1], mode='concat', axis=3, name='inception_4b_output')

    # 4c
    inception_4c_1_1 = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_1_1')
    inception_4c_3_3_reduce = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_3_3_reduce')
    inception_4c_3_3 = conv_2d(inception_4c_3_3_reduce, 256,  filter_size=3, activation='relu', name='inception_4c_3_3')
    inception_4c_5_5_reduce = conv_2d(inception_4b_output, 24, filter_size=1, activation='relu', name='inception_4c_5_5_reduce')
    inception_4c_5_5 = conv_2d(inception_4c_5_5_reduce, 64,  filter_size=5, activation='relu', name='inception_4c_5_5')
    inception_4c_pool = max_pool_2d(inception_4b_output, kernel_size=3, strides=1)
    inception_4c_pool_1_1 = conv_2d(inception_4c_pool, 64, filter_size=1, activation='relu', name='inception_4c_pool_1_1')
    inception_4c_output = merge([inception_4c_1_1, inception_4c_3_3, inception_4c_5_5, inception_4c_pool_1_1], mode='concat', axis=3, name='inception_4c_output')

    # 4d
    inception_4d_1_1 = conv_2d(inception_4c_output, 112, filter_size=1, activation='relu', name='inception_4d_1_1')
    inception_4d_3_3_reduce = conv_2d(inception_4c_output, 144, filter_size=1, activation='relu', name='inception_4d_3_3_reduce')
    inception_4d_3_3 = conv_2d(inception_4d_3_3_reduce, 288, filter_size=3, activation='relu', name='inception_4d_3_3')
    inception_4d_5_5_reduce = conv_2d(inception_4c_output, 32, filter_size=1, activation='relu', name='inception_4d_5_5_reduce')
    inception_4d_5_5 = conv_2d(inception_4d_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4d_5_5')
    inception_4d_pool = max_pool_2d(inception_4c_output, kernel_size=3, strides=1,  name='inception_4d_pool')
    inception_4d_pool_1_1 = conv_2d(inception_4d_pool, 64, filter_size=1, activation='relu', name='inception_4d_pool_1_1')
    inception_4d_output = merge([inception_4d_1_1, inception_4d_3_3, inception_4d_5_5, inception_4d_pool_1_1], mode='concat', axis=3, name='inception_4d_output')

    # 4e
    inception_4e_1_1 = conv_2d(inception_4d_output, 256, filter_size=1, activation='relu', name='inception_4e_1_1')
    inception_4e_3_3_reduce = conv_2d(inception_4d_output, 160, filter_size=1, activation='relu', name='inception_4e_3_3_reduce')
    inception_4e_3_3 = conv_2d(inception_4e_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_4e_3_3')
    inception_4e_5_5_reduce = conv_2d(inception_4d_output, 32, filter_size=1, activation='relu', name='inception_4e_5_5_reduce')
    inception_4e_5_5 = conv_2d(inception_4e_5_5_reduce, 128,  filter_size=5, activation='relu', name='inception_4e_5_5')
    inception_4e_pool = max_pool_2d(inception_4d_output, kernel_size=3, strides=1,  name='inception_4e_pool')
    inception_4e_pool_1_1 = conv_2d(inception_4e_pool, 128, filter_size=1, activation='relu', name='inception_4e_pool_1_1')
    inception_4e_output = merge([inception_4e_1_1, inception_4e_3_3, inception_4e_5_5, inception_4e_pool_1_1], axis=3, mode='concat')
    pool4_3_3 = max_pool_2d(inception_4e_output, kernel_size=3, strides=2, name='pool_3_3')

    # 5a
    inception_5a_1_1 = conv_2d(pool4_3_3, 256, filter_size=1, activation='relu', name='inception_5a_1_1')
    inception_5a_3_3_reduce = conv_2d(pool4_3_3, 160, filter_size=1, activation='relu', name='inception_5a_3_3_reduce')
    inception_5a_3_3 = conv_2d(inception_5a_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_5a_3_3')
    inception_5a_5_5_reduce = conv_2d(pool4_3_3, 32, filter_size=1, activation='relu', name='inception_5a_5_5_reduce')
    inception_5a_5_5 = conv_2d(inception_5a_5_5_reduce, 128, filter_size=5,  activation='relu', name='inception_5a_5_5')
    inception_5a_pool = max_pool_2d(pool4_3_3, kernel_size=3, strides=1,  name='inception_5a_pool')
    inception_5a_pool_1_1 = conv_2d(inception_5a_pool, 128, filter_size=1, activation='relu', name='inception_5a_pool_1_1')
    inception_5a_output = merge([inception_5a_1_1, inception_5a_3_3, inception_5a_5_5, inception_5a_pool_1_1], axis=3, mode='concat')

    # 5b
    inception_5b_1_1 = conv_2d(inception_5a_output, 384, filter_size=1, activation='relu', name='inception_5b_1_1')
    inception_5b_3_3_reduce = conv_2d(inception_5a_output, 192, filter_size=1, activation='relu', name='inception_5b_3_3_reduce')
    inception_5b_3_3 = conv_2d(inception_5b_3_3_reduce, 384,  filter_size=3, activation='relu', name='inception_5b_3_3')
    inception_5b_5_5_reduce = conv_2d(inception_5a_output, 48, filter_size=1, activation='relu', name='inception_5b_5_5_reduce')
    inception_5b_5_5 = conv_2d(inception_5b_5_5_reduce, 128, filter_size=5, activation='relu', name='inception_5b_5_5')
    inception_5b_pool = max_pool_2d(inception_5a_output, kernel_size=3, strides=1,  name='inception_5b_pool')
    inception_5b_pool_1_1 = conv_2d(inception_5b_pool, 128, filter_size=1, activation='relu', name='inception_5b_pool_1_1')
    inception_5b_output = merge([inception_5b_1_1, inception_5b_3_3, inception_5b_5_5, inception_5b_pool_1_1], axis=3, mode='concat')
    pool5_7_7 = avg_pool_2d(inception_5b_output, kernel_size=7, strides=1)
    pool5_7_7 = dropout(pool5_7_7, 0.4)

    # fc
    net = fully_connected(pool5_7_7, classes, activation='softmax')
    
    return net

In [9]:
def train(num_epochs=10, mode ='resnet32'):
    batch_size = 100
    classes = 2

    img_prep = tflearn.ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    img_aug = tflearn.ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_90degrees_rotation(rotations=[0, 1, 2, 3])
    img_aug.add_random_crop([75,75,2], padding=4)

    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', data_preprocessing=img_prep,data_augmentation=img_aug)

    Xtr, Ytr, Xte, ref = read_data()
    Xtr, Ytr = tflearn.data_utils.shuffle(Xtr, Ytr)
    Ytr = onehot_encoding(Ytr, classes)
    X, Y = Xtr[:1200], Ytr[:1200]
    testX, testY =Xtr[1200:1600], Ytr[1200:1600]
    
    
    if mode =='resnet32':
        network = tflearn.regression(resnet32(Input,classes),optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network,tensorboard_verbose=3, checkpoint_path='./tmp/')
        model.fit(X, Y, n_epoch=num_epochs, shuffle=True, validation_set=(testX, testY),
                  show_metric=True, batch_size=batch_size, run_id='resnet32')
        predict(model, Xte, ref, mode)
        
    elif mode =='inception_resnet_v2':        
        network = tflearn.regression(inception_resnet_v2(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network,tensorboard_verbose=3, checkpoint_path='./tmp/')
        model.fit(X, Y, n_epoch=num_epochs, shuffle=True, show_metric=True, validation_set=(testX, testY),
                  batch_size=batch_size, run_id='inception_resnet_v2')
        predict(model, Xte, ref, mode)
        
    elif mode=='resnext':
        network = tflearn.regression(resnext(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/', tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='resnext')
        predict(model, Xte, ref, mode)
        
    elif mode=='googlenet':
        network = regression(googlenet(Input,classes),  optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/', tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='googlenet')
        predict(model, Xte, ref, mode)
        
    elif mode=='vgg':
        network = regression(vgg(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/',tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='vgg')
        predict(model, Xte, ref, mode)
        
    else:
        print("Please Select Model in ['resnet32', 'inception_resnet_v2', 'resnext', 'googlenet','vgg']")
        
    def train(num_epochs=10, mode ='resnet32'):
    batch_size = 100
    classes = 2

    img_prep = tflearn.ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    img_aug = tflearn.ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_90degrees_rotation(rotations=[0, 1, 2, 3])
    img_aug.add_random_crop([75,75,2], padding=4)

    Input = tflearn.input_data(shape=[None, 75, 75, 2], dtype='float', data_preprocessing=img_prep,data_augmentation=img_aug)

    Xtr, Ytr, Xte, ref = read_data()
    Xtr, Ytr = tflearn.data_utils.shuffle(Xtr, Ytr)
    Ytr = onehot_encoding(Ytr, classes)
    X, Y = Xtr[:1200], Ytr[:1200]
    testX, testY =Xtr[1200:1600], Ytr[1200:1600]
    
    
    if mode =='resnet32':
        network = tflearn.regression(resnet32(Input,classes),optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network,tensorboard_verbose=3, checkpoint_path='./tmp/')
        model.fit(X, Y, n_epoch=num_epochs, shuffle=True, validation_set=(testX, testY),
                  show_metric=True, batch_size=batch_size, run_id='resnet32')
        predict(model, Xte, ref, mode)
        
    elif mode =='inception_resnet_v2':        
        network = tflearn.regression(inception_resnet_v2(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network,tensorboard_verbose=3, checkpoint_path='./tmp/')
        model.fit(X, Y, n_epoch=num_epochs, shuffle=True, show_metric=True, validation_set=(testX, testY),
                  batch_size=batch_size, run_id='inception_resnet_v2')
        predict(model, Xte, ref, mode)
        
    elif mode=='resnext':
        network = tflearn.regression(resnext(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/', tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='resnext')
        predict(model, Xte, ref, mode)
        
    elif mode=='googlenet':
        network = regression(googlenet(Input,classes),  optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/', tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='googlenet')
        predict(model, Xte, ref, mode)
        
    elif mode=='vgg':
        network = regression(vgg(Input,classes), optimizer='adam',loss='categorical_crossentropy')
        model = tflearn.DNN(network, checkpoint_path='./tmp/',tensorboard_verbose=3)
        model.fit(X, Y, n_epoch=num_epochs, validation_set=(testX, testY),show_metric=True,  batch_size=batch_size, 
                  shuffle=True, run_id='vgg')
        predict(model, Xte, ref, mode)
        
    else:
        print("Please Select Model in ['resnet32', 'inception_resnet_v2', 'resnext', 'googlenet','vgg']")
    
    model.save(mode)
    
def predict(model, test_data, ref, mode):
    #8424=8*81*13
    batch_size =104
    iteration = int(len(test_data)/batch_size)  #iter=81  
    with open("C:/Users/User/Desktop/kaggle/result{0}.txt".format(mode), "w") as f:
        f.write("id,is_iceberg\n")
        for i in range(iteration):
            pred = model.predict(test_data[i*batch_size: (i+1)*batch_size])
            for j in range(batch_size):
                data = "%s,%f\n" % (ref[i*batch_size+j], pred[j][1])
                f.write(data)

def ensemble(models, num_epochs = 20):
    
    for k in models: 
        models[k] = tf.Graph()
        with models[k].as_default():
            tflearn.init_graph()
            train(num_epochs, mode = k)
        tf.reset_default_graph()
    return model
    
        hypotheses[n_ensemble]=model.predict(testX)
        hypotheses_valid[n_ensemble]=model.predict(validX)
        n_ensemble+=1
    
    
tflearn.config.is_training (is_training=False, session=None)
# Retrieve variable responsible for managing training mode
training_mode = tflearn.get_training_mode()
# Define a conditional op
my_conditional_op = tf.cond(training_mode, if_yes_op, if_no_op)
# Set training mode to True
tflearn.is_training(True)
session.run(my_conditional_op)
if_yes_op
# Set training mode to False
tflearn.is_training(False)
session.run(my_conditional_op)
if_no_op
    
    
    
if __name__ == '__main__':
    models = {'resnet32': m1, 'vgg': m2, 'googlenet': m3, 'resnext': m4, 'inception_resnet_v2': m5}
    
    train(mode='resnet32', num_epochs=100)

Training Step: 11  | total loss: 0.63141 | time: 33.456s
| Adam | epoch: 001 | loss: 0.63141 - acc: 0.6456 -- iter: 1100/1200
Training Step: 12  | total loss: 0.62108 | time: 37.696s
| Adam | epoch: 001 | loss: 0.62108 - acc: 0.6926 | val_loss: 1.72386 - val_acc: 0.5400 -- iter: 1200/1200
--
INFO:tensorflow:C:\Users\User\Desktop\kaggle\tmp-12 is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected,reshape
from tflearn.layers.conv import conv_2d, max_pool_2d,highway_conv_2d
from tflearn.layers.normalization import local_response_normalization,batch_normalization
from tflearn.layers.estimator import regression
from tflearn.layers.merge_ops import merge
# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
import tflearn.data_utils as du

import tensorflow as tf
import numpy as np

def convnet(imgaug,drop_out = 0.5):
    network = input_data(shape=[None, 28, 28, 1], name='input', data_augmentation=imgaug)
    network = conv_2d(network, 32, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network) #try with batch norm
    network = conv_2d(network, 64, 5, activation='elu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = batch_normalization(network)
    network = conv_2d(network, 128, 5, activation='elu', regularizer="L2")
    network = batch_normalization(network)
    network = fully_connected(network, 512, activation='elu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 2048, activation='elu')
    network = dropout(network, drop_out)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    return network

def rnn(imgaug=None,drop_out = 0.5):
    rnn = input_data(shape=[None, 28, 28, 1], name='input', data_augmentation=imgaug)
    rnn  = reshape(rnn, [-1,28,28])
    rnn = tflearn.lstm(rnn, 256, return_seq=True)
    rnn = tflearn.lstm(rnn, 512)
    rnn = dropout(rnn,drop_out);
    rnn = fully_connected(rnn, 10, activation='softmax')
    rnn = regression(rnn, optimizer='adam', learning_rate=0.001,
                             loss='categorical_crossentropy', name='target')
    return rnn

def highnet(imgaug = None,drop_out = 0.5):
    highnet = input_data(shape=[None, 28, 28, 1], name='input', data_augmentation=imgaug)

    for i in range(3):
        for j in [3, 2, 1]:
            highnet = highway_conv_2d(highnet, 16, j, activation='elu')
        highnet = max_pool_2d(highnet, 2)
        highnet = batch_normalization(highnet)

    highnet = fully_connected(highnet, 128, activation='elu')
    highnet = dropout(highnet,drop_out);
    highnet = fully_connected(highnet, 256, activation='elu')
    highnet = dropout(highnet,drop_out);
    highnet = fully_connected(highnet, 10, activation='softmax')
    highnet = regression(highnet, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')
    return highnet

def resnet(imgaug=None, drop_out=0.5):
    # Building Residual Network
    net = input_data(shape=[None, 28, 28, 1], name='input', data_augmentation=imgaug)
    net = tflearn.conv_2d(net, 64, 3, activation='elu', bias=False)
    # Residual blocks
    net = tflearn.residual_bottleneck(net, 3, 16, 64)
    net = tflearn.residual_bottleneck(net, 1, 32, 128, downsample=True)
    net = tflearn.residual_bottleneck(net, 2, 32, 128)
    net = tflearn.residual_bottleneck(net, 1, 64, 256, downsample=True)
    net = tflearn.residual_bottleneck(net, 2, 64, 256)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'elu')
    net = tflearn.global_avg_pool(net)
    # Regression
    net = tflearn.fully_connected(net, 10, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy',
                             learning_rate=0.001, name='target')
    return net

#X, Y, testX, testY = mnist.load_data(one_hot=True)
dataset = mnist.read_data_sets(one_hot=True)
X = dataset.train.images
Y = dataset.train.labels
testX = dataset.test.images
testY = dataset.test.labels
validX=dataset.validation.images
validY=dataset.validation.labels

X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])
validX = validX.reshape([-1, 28, 28, 1])

X, mean = du.featurewise_zero_center(X)
testX = du.featurewise_zero_center(testX, mean)
validX = du.featurewise_zero_center(validX, mean)

ensemble=11   #Total no of ensemble models : 11
n_ensemble=0   #n_semble counter
n_sample=10000 #Number of validation set from training set
validX = np.concatenate((validX,X[0:n_sample]),axis=0)
validY = np.concatenate((validY,Y[0:n_sample]),axis=0)
hypotheses = np.zeros((ensemble,testY.shape[0],10))
hypotheses_valid = np.zeros((ensemble,validY.shape[0],10))

# Example: pictures of 32x32
imgaug = tflearn.ImageAugmentation()
imgaug.add_random_crop((28, 28), 4)
imgaug.add_random_rotation (max_angle=7.5)
imgaug.add_random_blur(0.3)

# Training graphs
conv_m1 = tf.Graph()
conv_m2 = tf.Graph()
conv_m3 = tf.Graph()
conv_m4 = tf.Graph()
conv_m5 = tf.Graph()
rnn_m1 = tf.Graph()
rnn_m2 = tf.Graph()
rnn_m3 = tf.Graph()
high_m1 = tf.Graph()
high_m2 = tf.Graph()
high_m3 = tf.Graph()

#res_m1 = tf.Graph()


global_epoch=200

#ensemble trial
with conv_m1.as_default():
    tf.set_random_seed(777) #no reproducibility...T.T
    tflearn.init_graph(seed=777)
    network1 = convnet(imgaug=None,drop_out=0.7)
    model = tflearn.DNN(network1, tensorboard_verbose=0)
    model.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='convnet_mnist1', batch_size=512)
    model.save('convnet1')
    hypotheses[n_ensemble]=model.predict(testX)
    hypotheses_valid[n_ensemble]=model.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()
with conv_m2.as_default():
    tf.set_random_seed(777) 
    tflearn.init_graph(seed=777)
    network2 = convnet(imgaug=imgaug,drop_out=0.6)
    model2 = tflearn.DNN(network2, tensorboard_verbose=0)
    model2.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='convnet_mnist1', batch_size=512)
    model2.save('convnet2')
    hypotheses[n_ensemble]=model2.predict(testX)
    hypotheses_valid[n_ensemble]=model2.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()
with conv_m3.as_default():
    tf.set_random_seed(779)
    tflearn.init_graph(seed=779)
    network3 = convnet(imgaug,drop_out=0.6)
    model3 = tflearn.DNN(network3, tensorboard_verbose=0)
    model3.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch/1.5),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='convnet_mnist3', batch_size=512)
    model3.save('convnet3')
    hypotheses[n_ensemble]=model3.predict(testX)
    hypotheses_valid[n_ensemble]=model3.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()
with conv_m4.as_default():
    tf.set_random_seed(780)
    tflearn.init_graph(seed=780)
    network4 = convnet(imgaug,drop_out=0.5)
    model4 = tflearn.DNN(network4, tensorboard_verbose=0)
    model4.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch/2),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='convnet_mnist4', batch_size=512)
    model4.save('convnet4')
    hypotheses[n_ensemble]=model4.predict(testX)
    hypotheses_valid[n_ensemble]=model4.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()
with conv_m5.as_default():
    tf.set_random_seed(781)
    tflearn.init_graph(seed=781)

    network5 = convnet(imgaug,drop_out=0.4)
    model5 = tflearn.DNN(network5, tensorboard_verbose=0)

    model5.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='convnet_mnist5', batch_size=512)
    model5.save('convnet5')
    hypotheses[n_ensemble]=model5.predict(testX)
    hypotheses_valid[n_ensemble]=model5.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()


with rnn_m1.as_default():
    tf.set_random_seed(782)
    tflearn.init_graph(seed=782)

    rnn_network1 = rnn(None,drop_out=0.7)
    rnn1 = tflearn.DNN(rnn_network1, tensorboard_verbose=0)
    rnn1.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='rnn_mnist1',batch_size=512)
    rnn1.save('rnn1')
    hypotheses[n_ensemble]=rnn1.predict(testX)
    hypotheses_valid[n_ensemble]=rnn1.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()

with rnn_m2.as_default():
    tf.set_random_seed(783)
    tflearn.init_graph(seed=783)

    rnn_network2 = rnn(imgaug,drop_out=0.6)
    rnn2 = tflearn.DNN(rnn_network2, tensorboard_verbose=0)
    rnn2.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch/2),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='rnn_mnist2',batch_size=512)
    rnn2.save('rnn2')
    hypotheses[n_ensemble]=rnn2.predict(testX)
    hypotheses_valid[n_ensemble]=rnn2.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()

with rnn_m3.as_default():
    tf.set_random_seed(784)
    tflearn.init_graph(seed=784)

    rnn_network3 = rnn(imgaug,drop_out=0.5)
    rnn3 = tflearn.DNN(rnn_network3, tensorboard_verbose=0)
    rnn3.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch/1),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='rnn_mnist3',batch_size=512)
    rnn3.save('rnn3')
    hypotheses[n_ensemble]=rnn3.predict(testX)
    hypotheses_valid[n_ensemble]=rnn3.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()


with high_m1.as_default():
    tf.set_random_seed(785)
    tflearn.init_graph(seed=785)

    highnet_network1=highnet(None,drop_out=0.7)
    highnet1 = tflearn.DNN(highnet_network1, tensorboard_verbose=0)
    highnet1.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='highway_mnist1',batch_size=512)
    highnet1.save('highway1')
    hypotheses[n_ensemble]=highnet1.predict(testX)
    hypotheses_valid[n_ensemble]=highnet1.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()

with high_m2.as_default():
    tf.set_random_seed(786)
    tflearn.init_graph(seed=786)

    highnet_network2=highnet(imgaug,drop_out=0.6)
    highnet2 = tflearn.DNN(highnet_network2, tensorboard_verbose=0)
    highnet2.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='highway_mnist2',batch_size=512)
    highnet2.save('highway2')
    hypotheses[n_ensemble]=highnet2.predict(testX)
    hypotheses_valid[n_ensemble]=highnet2.predict(validX)

    n_ensemble+=1
tf.reset_default_graph()

with high_m3.as_default():
    tf.set_random_seed(787)
    tflearn.init_graph(seed=787)

    highnet_network3=highnet(imgaug,drop_out=0.5)
    highnet3 = tflearn.DNN(highnet_network3, tensorboard_verbose=0)
    highnet3.fit({'input': X}, {'target': Y}, n_epoch=int(global_epoch),
               validation_set=({'input': validX[:5000]}, {'target': validY[:5000]}),
               snapshot_step=55000, show_metric=True, run_id='highway_mnist3',batch_size=512)
    highnet3.save('highway3')
    hypotheses[n_ensemble]=highnet3.predict(testX)
    hypotheses_valid[n_ensemble]=highnet3.predict(validX)
    n_ensemble+=1
tf.reset_default_graph()

#End of training

print("Optimization finished")

hypotheses_valid = hypotheses_valid.transpose([1,0,2])
hypotheses = hypotheses.transpose([1,0,2])
hypothesis = np.zeros((testY.shape[0],10))

#Ensemble weight training : using Validation set
ens=tf.Graph()
with ens.as_default():
    ensemble_net = input_data(shape=[None,n_ensemble, 10], name='input')
    ensemble_net = fully_connected(ensemble_net, 10, activation='softplus')
    ensemble_net = regression(ensemble_net, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    ens_model = tflearn.DNN(ensemble_net, tensorboard_verbose=0)
    ens_model.fit({'input': hypotheses_valid}, {'target': validY}, n_epoch=20,
                  snapshot_step=5000, show_metric=False, run_id='ensemble',batch_size=1000)

    hypothesis = ens_model.predict(hypotheses)
    ens_model.save('ens_model')


correct_prediction = np.equal(np.argmax(hypothesis,1), np.argmax(testY, 1))
accuracy_ens = np.mean(correct_prediction)

print("Final Accuracy :", accuracy_ens)